In [1]:
import re
import os 
import datetime
from math import log
import csv
import pprint
import math
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd
from pandas.io.parsers import read_csv
pd.set_option('max_colwidth',120)
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
from __future__ import division

%matplotlib inline

pp = pprint.PrettyPrinter()

In [ ]:
#bashcommand='pwd'

In [2]:
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=10)

In [3]:
indices = es.cat.indices(index=['jobs_archive_2016-05-14','jobs_archive_2016-05-15','jobs_archive_2016-05-16'], h="index", request_timeout=10).split('\n')
indices = sorted(indices)
indices = [x.replace(" ", "") for x in indices if x != '']

In [28]:
print (indices)

['jobs_archive_2016-05-14', 'jobs_archive_2016-05-15', 'jobs_archive_2016-05-16']


In [23]:
ss={
 "query": { "prefix": { "computingsite": "IN2P3-CC" } 
}, "_source": ["pandaid","batchid","computingsite","computingelement","corecount","creationtime","starttime","endtime",
   "cpuconsumptiontime","wall_time","modificationhost","maxrss","maxpss"]
#    ,"size": 5
    }



#ss= {"query": { "match_all": {} },"size": 10000} 
print (ss)



{'query': {'prefix': {'computingsite': 'IN2P3-CC'}}, '_source': ['pandaid', 'batchid', 'computingsite', 'computingelement', 'corecount', 'creationtime', 'starttime', 'endtime', 'cpuconsumptiontime', 'wall_time', 'modificationhost', 'maxrss', 'maxpss']}


In [ ]:
#res = es.search(index='jobs_archive_2016-06-26', body=ss, filter_path=['hits.hits._source'],request_timeout=12000)

In [24]:
res=es.search(index=indices, body=ss,request_timeout=12000)
#print (res)
len(res)
#print (res)

4

In [ ]:
 #res.get('hits').get('hits')

In [25]:
scroll=scan(es,index=indices, query=ss,scroll=u'30m', raise_on_error=True, preserve_order=False, size=200000,request_timeout=300)

In [ ]:
c=1
list2=[]
for name in scroll:
 a=name.get('_source')
 list2.append(a)
 c=c+1
print (c)
type(list2)
dfu = pd.DataFrame(list2)
 
#dfu.batchid.count()
dfu.head()
 
 

1


""


In [20]:
dfu.groupby(dfu.computingelement).computingsite.count()

computingelement
IN2P3-CC-T3_MCORE             255
IN2P3-CC-T3_VM01             1151
IN2P3-CC-all-ce-sge-long    39006
IN2P3-CC_MCORE_HIMEM         6491
IN2P3-CC_VVL                 1567
cccreamceli01.in2p3.fr       3172
cccreamceli05.in2p3.fr        647
cccreamceli07.in2p3.fr          6
Name: computingsite, dtype: int64

In [21]:
dfu.groupby(dfu.computingsite).count()

,batchid,computingelement,corecount,cpuconsumptiontime,creationtime,endtime,maxpss,maxrss,modificationhost,pandaid,starttime,wall_time
computingsite,,,,,,,,,,,,
IN2P3-CC,39006,39651,39748,39753,39753,39753,38934,38934,39753,39753,39006,39753
IN2P3-CC-T3_MCORE,255,255,255,255,255,255,249,249,255,255,255,255
IN2P3-CC-T3_VM01,1151,1157,1149,1157,1157,1157,1148,1148,1157,1157,1151,1157
IN2P3-CC_MCORE_HIMEM,6490,9663,9663,9663,9663,9663,6456,6456,9663,9663,6491,9663
IN2P3-CC_VVL,1567,1569,1569,1569,1569,1569,1565,1565,1569,1569,1567,1569


In [ ]:


#rint (res.get('hits').get('hits'))
#list.head()
#res[0]
#res['total']

In [ ]:
dfu.head()

In [ ]:
 dfu.groupby(['computingsite']).count()

In [ ]:
b=dfu.avgpss/1024/1024
b.hist(bins=100)

In [ ]:
dfu.memory_usage()

In [27]:
dfu.to_csv('./ev_example_161516_all.csv')